# Criação de dados

A proposta deste notebook é criar um dataframe de dados mockados contendo as seguintes colunas:
- id
- data
- usuário
- empresa

In [1]:
import pandas as pd
from faker import Faker
import uuid
from datetime import datetime
from random import choice

### Imports
- **pandas**: para criar uma tabela
- **faker**: para criar os dados
- **uuid**: para chaves primárias unicas
- **datetime e random**: complementares

In [2]:
# Instanciando o Faker
fake = Faker(["pt_BR"])

In [3]:
# Quantidade de registros na tabela

lines = 1_000_000

In [4]:
# Criação de dados 

base_company_list = [fake.unique.company_id() for _ in range(int(1_000))] # 1k emrpesas
base_user_list = [fake.unique.email() for _ in range(int(20_000))] # 20k de usuários

# para um contexto mais lógico, onde um usuário sempre pertencer à uma mesma empresa
user_company = [(base_user, choice(base_company_list)) for base_user in base_user_list]
full_user_company = [choice(user_company) for _ in range(lines)]
 
# Criação dos dados
data = {
    "id": [uuid.uuid4() for _ in range(lines)],
    "insert_date": [fake.date_time_between(start_date=datetime(2023, 10, 1)) for _ in range(lines)],
    "username": [user[0] for user in full_user_company],
    "company": [company[1] for company in full_user_company],
}


In [5]:
# Criação do dataframe
df = pd.DataFrame(data)

In [6]:
# Apenas vendo como ficou a tabela
df.head()

,id,insert_date,username,company
0,ecaa3a99-a802-40ab-8e09-8f34b87cc759,2023-11-01 12:14:02,freitasana-vitoria@example.org,63728149000101
1,7b33cd0c-9fdb-48ef-bf00-98792016990e,2023-10-07 02:15:48,benjamin45@example.com,62491075000170
2,001f87c1-f72e-4e75-8b32-ebdbd47ac4a0,2023-10-12 01:17:27,bernardoda-cunha@example.org,59843210000194
3,f29ccb2f-3a05-4575-b49e-864ee8c1e738,2023-11-02 13:41:44,qgomes@example.net,09368152000116
4,83f47de8-3a17-499f-8355-573091e08146,2023-10-19 12:51:30,vfreitas@example.org,46057892000168


In [7]:
# Vendo os tipos das colunas
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 4 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   id           1000000 non-null  object        
 1   insert_date  1000000 non-null  datetime64[ns]
 2   username     1000000 non-null  object        
 3   company      1000000 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 30.5+ MB


In [ ]:
# Gerando o arquivo SQL

# Este foi o filename que usei ao rodar no meu pc para criar o script
filename = "../docker_databases/init.sql"

# Estou deixando este apenas para que possa executar no seu computador sem comprometer o resultado
# das analises que realizei.
# filename = "../docker_databases/init_avaliacao.sql"

with open(filename, "w") as f:
    f.write("\n\n-- Criação da tabela")
    f.write("""
    CREATE TABLE t_ponto (
          id VARCHAR(255) PRIMARY KEY,
          insert_date TIMESTAMP,
          username VARCHAR(255),
          company VARCHAR(255)
    );\n
    """)
    
    f.write("\n\n-- Exportação dos dados\n\n")
    for i in df.index:
        f.write(f"INSERT INTO t_ponto (id, insert_date, username, company) VALUES ('{str(df.iloc[i, 0])}', '{df.iloc[i, 1]}', '{df.iloc[i, 2]}', '{df.iloc[i, 3]}');\n")

In [ ]:
print("Fim")

## Conclusão

foi gerado um script sql para que possa ser instanciado o container com o banco ja contendo as informações. Sempre que o container for executado no composer, ele ja irá conter os dados.